# Sound Level Meter Functionality

### Obtain Data and View the Meta Data

In [1]:
import pyquist as pq
import numpy as np

In [2]:
meta, data = pq.io.read_tdms('s3://signals.aercoustics.com/vibration/train/surface_radio.tdms')
meta

{'file type': 'tdms',
 'manufacturer': 'National Instruments',
 'names': {'Raw Data': ['Acceleration']},
 'root properties': {},
 'group properties': {'Raw Data': {}},
 'channel properties': {'Raw Data': {'Acceleration': {'wf_increment': 0.0001953125,
    'wf_start_offset': 0}}},
 'format': {'sample rate': 5120.0,
  'sample size': 32,
  'number of channels': 1,
  'number of samples': 207222},
 'timestamp': {'start time': None, 'duration': '0:00:40.473047'}}

In [3]:
accel = data['Raw Data']['Acceleration']
fs = meta['format']['sample rate']
time_steps, accel = pq.time_steps(accel, fs)
time_steps_lev, levels = pq.level(accel, fs, time_interval=0.125, overlap=0.9, ref=pq.REF_ACCEL)

### Plot the Raw Acceleration Data

In [4]:
plot1 = pq.vis.plot_grid_channels(time_steps, accel, figure_options=dict(title='Raw Acceleration Data', x_axis_label='Time [s]', y_axis_label='Acceleration [m/s^2]'))

### Plot the Acceleration Levels

In [5]:
plot2 = pq.vis.plot_grid_channels(time_steps_lev, levels,
                                  figure_options=dict(title='Acceleration Data in Levels', x_axis_label='Time [s]', y_axis_label='Acceleration [dB ref 10μm/s^2]'),
                                  color='red', line_width=2)

### Plot a Histogram of the Frequency of Acceleration Levels

In [6]:
plot3 = pq.vis.plot_histogram(levels, percentile_hover=False,
                              figure_options= dict(title='Histogram of Frequency of Acceleration Levels', x_axis_label='Acceleration [dB ref 10μm/s^2]', y_axis_label='Count'),
                              histogram_options=dict(bins=25))

### Calculate and Plot Statistics

In [7]:
stats = pq.stats(levels)
L10, L50, L90 = stats['L10'], stats['L50'], stats['L90']

In [8]:
L10_arr = L10 * np.ones([len(time_steps_lev)])
L50_arr = L50 * np.ones([len(time_steps_lev)])
L90_arr = L90 * np.ones([len(time_steps_lev)])

In [9]:
stacked = np.stack((levels, L10_arr, L50_arr, L90_arr))

In [10]:
plot4 = pq.vis.plot_overlaid_channels(time_steps_lev, stacked, 
                                      legend= ['Acceleration Level', 'L10= %s [dB]'%(np.round(L10, 2)), 'L50= %s [dB]'%(np.round(L50, 2)), 'L90= %s [dB]'%(np.round(L90,2))],
                                      figure_options=dict(title='Acceleration Data in Levels', x_axis_label='Time [s]', y_axis_label='Acceleration [dB ref 10μm/s^2]'),
                                      color=('red', 'blue', 'green', 'yellow'), line_width=(2,1,1,1))

# Decibel Math

### Get the Pressure and Decibel Data

In [11]:
columns, table = pq.io.read_excel("C:\\Users\\jnam\\Desktop\\uofmo\\Basics of Acoustics.xlsx", 
                                  as_dataframe= False, row_labels= False, column_labels= True, 
                                  sheet_name='Sample dB conversion values')
pascals = table[:,np.where(columns=='Pa')].squeeze()
decibels = table[:,np.where(columns=='dB')].squeeze()

### Conversions

In [12]:
# Pascals to Decibels
converted_pa = np.round(pq.dB_from_unit(pascals, ref=pq.REF_SPL), 2)
print('Pascals:', *pascals, sep='  ')
print('Decibels:', *converted_pa)

Pascals:  20  1000  20000
Decibels: 120.0 153.98 180.0


In [13]:
# Decibels to Pascals
converted_dB = np.round(pq.unit_from_dB(decibels, ref=pq.REF_SPL),5)
print('Decibels:', *decibels)
print('Pascals:', *converted_dB, sep='  ')

Decibels: 80 90 120
Pascals:  0.2  0.63246  20.0


### Decibel Summation, Difference and Average

In [28]:
# Decibel Summation
dB_sum = np.round(pq.dB_sum(decibels),4)
print('%s dB + %s dB + %s dB = %s dB' %(decibels[0], decibels[1], decibels[2], dB_sum))

80 dB + 90 dB + 120 dB = 120.0048 dB


In [31]:
# Decibel Difference
dB_diff = np.round(pq.dB_sub(decibels[1], decibels[0]), 4)
print('%s dB - %s dB = %s dB' %(decibels[1], decibels[0], dB_diff))

90 dB - 80 dB = 89.5424 dB


In [36]:
# Decibel Average
dB_avg = np.round(pq.dB_avg(decibels), 4)
print('Average of %s dB, %s dB, and %s dB = %s dB'  %(decibels[0], decibels[1], decibels[2], dB_avg))

Average of 80 dB, 90 dB, and 120 dB = 115.2336 dB
